In [ ]:
%%javascript
require.config({
    paths: {
        DT: '//cdn.datatables.net/1.10.19/js/jquery.dataTables.min',
    }
});

In [2]:
import pprint
import sys
import spotipy
import spotipy.util as util
import simplejson as json
from tabulate import tabulate


############################
# Spotify API Credentials  #
############################
username = 'fas3i0ct87zm1sbtjxsdne4r7'

scope = 'playlist-read-collaborative user-library-read user-top-read user-read-currently-playing user-read-playback-state user-modify-playback-state streaming app-remote-control playlist-modify-public playlist-read-private playlist-modify-private user-library-modify user-library-read'
client_id = ''
client_secret = ''
redirect_uri = 'http://google.com/'

deviceID = ''

############################
# Genious API Credentials  #
############################
ACCESS_TOKEN = ''
headers = {'Authorization':'Bearer ' + ACCESS_TOKEN }


In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib
import pandas as pd
import vlc

def make_clickable(val):
    return vlc.MediaPlayer(val)


def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_token():
    return util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    
def get_audio_features(a_song):
    token = get_token()
    if token:
        sp = spotipy.Spotify(auth=token)
        return sp.audio_features(a_song)[0]

def get_song_info(track):
    token = get_token()
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False

        song = track['name'] # Get current song name
        artist = track['artists'][0]['name'] # Get current song artist 
        preview_url = track['preview_url']
        
        audio_features = get_audio_features(track['id']) # Get song features
        
        # Prepare headers and table values
        headers = ['Song','Artist','preview_url']
        table_val = [song, artist, preview_url]
        for i in audio_features:
            if type(audio_features[i]) is float:
                headers.append(str(i).capitalize())
                table_val.append(audio_features[i])
        
        return pd.DataFrame([table_val], columns=headers) 
    
def get_curr_song_info():
    token = get_token()
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        curr_playing = sp.current_playback()

        ##########################################
        # Current Song Info
        ##########################################
        curr_playing = sp.current_playback()  # Get current song 
        song = curr_playing['item']['name'] # Get current song name
        artist = curr_playing['item']['artists'][0]['name'] # Get current song artist 
        
        audio_features = get_audio_features(curr_playing['item']['id']) # Get song features
        
        # Prepare headers and table values
        headers = ['Song','Artist']
        table_val = [song, artist]
        for i in audio_features:
            if type(audio_features[i]) is float:
                headers.append(str(i).capitalize())
                table_val.append(audio_features[i])
        
        return pd.DataFrame([table_val], columns=headers) 

def get_lyrics(search_str):
    
    # Prepare
    geni_search_base = 'http://api.genius.com/search?q='
    search_str = urllib.parse.quote(search_str) # Encode search str to be url friendly
    search_url = geni_search_base + search_str # Final search url
    search_res = requests.get(search_url, headers=headers) # Get response
    
    try:
        # Get soup
        soup_link = search_res.json()['response']['hits'][0]['result']['url']
        data = requests.get(soup_link)
        soup = BeautifulSoup(data.text, 'html.parser')
        lyrics = soup.find_all('div', class_='lyrics')
        
        # Print lyrics
        print('---------')
        print('Search Genious: \n', 'https://genius.com/search?q=' + search_str)
        print('---------')
        for i in lyrics:
            print(i.find('p').text)
            
    except IndexError:
        # handle IndexError exception
        print('Oops, No lyrics found! Try Search manually...')
    except (TypeError, ZeroDivisionError):
       # handle multiple exceptions
       # TypeError and ZeroDivisionError
        pass
    except:
        # handle all other exceptions
        pass

    

In [ ]:
sp.devices()['devices']

In [ ]:
everywhere = sp.devices()['devices'][0]['id']
iphone =  sp.devices()['devices'][2]['id']
web = sp.devices()['devices'][3]['id']

In [ ]:
##########################################
# Go to Previous Song
##########################################

sp = spotipy.Spotify(auth=get_token()) # Create spotify object
sp.trace = False # Disable trace
sp.previous_track(device_id=web) # Play

In [ ]:
##########################################
# Go to Next Song
##########################################
sp = spotipy.Spotify(auth=get_token()) # Create spotify object
sp.trace = False # Disable trace
sp.next_track(device_id=web) # Play

In [ ]:
##########################################
# Current Song 
##########################################
from IPython.display import display, HTML

sp = spotipy.Spotify(auth=get_token()) # Create spotify object

if not sp.current_user_playing_track()['is_playing']:
    sp.start_playback(device_id=web)
else:
    song_info = get_curr_song_info()
    song = str(song_info['Song'].values).strip("['']")
    artist = str(song_info['Artist'].values).strip("['']")
    print(song,'//', artist)
    display(HTML(song_info.to_html()))
    get_lyrics('下山-要不要买菜') # Print lyrics

In [ ]:
##########################################
# TOP 50 Songs
##########################################
from IPython.display import display, HTML
token = get_token()
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    results = sp.current_user_top_tracks(time_range='short_term')
    tracks = results['items']
    song_list = pd.DataFrame()
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        
    for i, item in enumerate(tracks):
        song_list = pd.concat([song_list, get_song_info(item)],ignore_index=True, sort=True)
    
    columns = ['Song', 'Artist', 'Danceability', 'Energy', 'Instrumentalness',
       'Liveness', 'Loudness',  'Speechiness', 'Acousticness', 'Valence', 'Tempo','preview_url']
    song_list = song_list.reindex(columns, axis="columns")
    
    song_list['preview_url'] = song_list['preview_url'].apply(make_clickable)
    


In [30]:
list_tbl = HTML(song_list.to_html(table_id='song_list', float_format='%.2f'))
list_tbl

,Song,Artist,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Acousticness,Valence,Tempo,preview_url
0,下山,要不要买菜,0.63,0.73,0.00,0.28,-6.19,0.11,0.76,0.61,163.97,<vlc.MediaPlayer object at 0x1117bc4a8>
1,everything i wanted,Billie Eilish,0.70,0.23,0.66,0.11,-14.45,0.10,0.90,0.24,120.01,<vlc.MediaPlayer object at 0x111447eb8>
2,On The Floor,Jennifer Lopez,0.73,0.78,0.00,0.07,-5.20,0.05,0.11,0.59,130.00,<vlc.MediaPlayer object at 0x111447a58>
3,Look At Her Now,Selena Gomez,0.56,0.56,0.03,0.30,-7.91,0.27,0.11,0.55,154.35,<vlc.MediaPlayer object at 0x1114479b0>
4,Blackspots,65daysofstatic,0.18,0.84,0.82,0.38,-6.22,0.07,0.01,0.14,84.83,<vlc.MediaPlayer object at 0x111447be0>
5,Mean,Taylor Swift,0.57,0.76,NaN,0.22,-3.99,0.05,0.45,0.79,163.97,<vlc.MediaPlayer object at 0x11ee5b940>
6,Applause - Fareoh Remix,Lady Gaga,0.66,0.53,0.00,0.09,-5.95,0.04,0.03,0.35,127.98,<vlc.MediaPlayer object at 0x11ee5b978>
7,Sandstorm,Darude,0.52,0.98,0.99,0.03,-7.13,0.04,0.02,0.50,136.03,<vlc.MediaPlayer object at 0x11ee5b908>
8,Guiding Light (feat. Sanna Martinez),QUIX,0.54,0.86,0.00,0.15,-3.25,0.05,0.35,0.38,145.12,<vlc.MediaPlayer object at 0x11ee5b898>
9,Only You,Cheat Codes,0.61,0.74,NaN,0.10,-4.79,0.17,0.08,0.22,108.05,<vlc.MediaPlayer object at 0x11ee5b7b8>


In [33]:
%%javascript
// the 'DT' here is the key in the `require.config` call
// make sure it matches the one from abovehttp://localhost:8888/notebooks/sp.ipynb#
window.executePython = function(python) {
    return new Promise((resolve, reject) => { 
        var callbacks = {
            iopub: {
                output: function(data) {resolve(data.content.text)}
            }
        };
        Jupyter.notebook.kernel.execute(`${python}`, callbacks);    
    });
}

function stop_playing(){
    var track_num = window.executePython('len(song_list)');
    console.log(track_num);
    for(var i=0; i < track_num; i++){
        var code = 'song_list["preview_url"][' + i + '].stop()';
        window.executePython( code );
    }
}
  

// subscriber function
function subscriber(mutations) {
    if (mutations){
//         console.log(mutations);
         $("#song_list > tbody > tr").click(function(){
            stop_playing();
            console.log($(this).find("th").text());
             
            $(this).toggleClass("playing");
            if ( $( this ).hasClass( "playing" ) ){
                var code = 'song_list["preview_url"][' + $(this).find("th").text() + '].play()';
            } else {
                var code = 'song_list["preview_url"][' + $(this).find("th").text() + '].stop()';
            }

             window.executePython( code );
        });
    }
}
// create observer
const observer = new MutationObserver(subscriber);

// by default all false
// you can pick as many as you want, but at least one of - attributes, characterData, or childList
const observerConfig = {
  childList: true, // target childs will be observed | on add/remove
};

require(["DT"], function(DT) {
    $(document).ready( function() {
        $("#song_list").DataTable();
        
        const observe_target = document.querySelector("#song_list > tbody"); // set observer target to monitor structure changes
        console.log(observe_target);
        observer.observe(observe_target,observerConfig);
        
        $("#song_list > tbody > tr").click(function(){
                stop_playing();
                console.log($(this).find("th").text());

                $(this).toggleClass("playing");
                if ( $( this ).hasClass( "playing" ) ){
                    var code = 'song_list["preview_url"][' + $(this).find("th").text() + '].play()';
                } 
                if ( !$( this ).hasClass( "playing" ) ) {
                    var code = 'song_list["preview_url"][' + $(this).find("th").text() + '].stop()';
                }

                window.executePython( code );
            });

        
    })
});


<IPython.core.display.Javascript object>

In [29]:
%%javascript
$("#song_list").DataTable();
stop_playing();

<IPython.core.display.Javascript object>

In [ ]:
%%javascript
IPython.keyboard_manager.command_shortcuts.add_shortcut("Ctrl-Shift-Alt-h", {
    help : 'Clear all output',
    help_index : 'zzz',
    handler : function() { 
        // to clear all the outputs
        IPython.notebook.clear_all_output();         
        return false;
    }}
);